In [40]:
import keras.backend as K
from keras.layers import Dense, MaxPooling2D, BatchNormalization, Input
from keras.layers import Dropout, Conv2D, MaxPool2D, GlobalAvgPool2D, GlobalMaxPool2D, Lambda
from keras.models import Sequential, Model

In [30]:
'this is the cnn block used in siamese network'
def build_siamese_network(filters, kernel_size, d_dim_rep, input_shape = (160,160,3)):
    input_ = Input(shape = input_shape)
    
    #define cnnm layers Using Functional Model
    x = Conv2D(filters, kernel_size, strides = 1, padding = 'valid', activation = 'relu')(input_)
    x = MaxPool2D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    
    #replicating the same structure
    x = Conv2D(filters, kernel_size, strides = 1, padding = 'valid', activation = 'relu')(x)
    x = MaxPool2D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    
    #replicating the same structure
    x = Conv2D(filters, kernel_size, strides = 1, padding = 'valid', activation = 'relu')(x)
    x = MaxPool2D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    
    #what we want to return here is a d dim vector 
    x = GlobalAvgPool2D()(x)
    x = Dense(units = 128, activation = 'relu',kernel_initializer='he_normal')(x)
    x = Dropout(0.4)(x)
    output = Dense(units = d_dim_rep, activation = 'relu')(x)
    
    siamese_model = Model(inputs = [input_], outputs = [output])
    siamese_model.summary()
    return siamese_model

cnn_block(32, 3, 64 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 160, 160, 3)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 158, 158, 32)      896       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 79, 79, 32)        0         
_________________________________________________________________
batch_normalization_52 (Batc (None, 79, 79, 32)        128       
_________________________________________________________________
dropout_62 (Dropout)         (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 77, 77, 32)        9248      
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 38, 38, 32)        0         
__________

### siamese model architecture

In [43]:
def euclidean_dist(vects):
    (feat1, feat2) = vects
    squared_sum = K.sum(K.square(feat1 - feat2), axis = 1, keepdims = True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(squared_sum, K.epsilon()))#epsilon is a very small value

#configure parameters
input_shape = (160,160,3)
batch_size = 32

### defining inputs
img1 = Input(input_shape)
img2 = Input(input_shape)

#instantiate network
feature_extractor = build_siamese_network(32,2,64)

feat1 = feature_extractor(img1)
feat2 = feature_extractor(img2)
# The Lambda layer exists so that arbitrary TensorFlow functions
# can be used when constructing Sequential and Functional API
# models. Lambda layers are best suited for simple operations or
# quick experimentation.
distance = Lambda(euclidean_dist)([feat1, feat2]) #input and output of lambda is tensors numpy is not allowed
output = Dense(1, activation='sigmoid')(distance)

model = Model([img1, img2], output)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_64 (InputLayer)        (None, 160, 160, 3)       0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 159, 159, 32)      416       
_________________________________________________________________
max_pooling2d_86 (MaxPooling (None, 79, 79, 32)        0         
_________________________________________________________________
batch_normalization_86 (Batc (None, 79, 79, 32)        128       
_________________________________________________________________
dropout_107 (Dropout)        (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 78, 78, 32)        4128      
_________________________________________________________________
max_pooling2d_87 (MaxPooling (None, 39, 39, 32)        0         
__________

In [ ]:
def load_data(width , height,train):
    'function to load data from the disk'
    if train == True:
        path = '5-celebrity-faces-dataset/faces_train'#train path
    else:
        path = '5-celebrity-faces-dataset/faces_val'#test path
        
    celeb_name_fols = os.listdir(path)
    arr = []
    label =[]
    celeb_dict = dict(zip(os.listdir('5-celebrity-faces-dataset/faces_train'), list(range(len(celeb_name_fols))) ))
    i = 0
    for fold in celeb_name_fols:
        for image in os.listdir(path + '/' + fold):
            label.append(celeb_dict[fold]) #label for celeb
            img = cv2.imread(path + '/' + fold + '/'+ image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (width, height))
            arr.append(img)
            i += 1
    print('the number of images loaded {}'.format(int(i)) )
    return np.array(arr), np.array(label), celeb_dict

In [ ]:
def make_pairs(images, labels, samples): #samples can incrase the number of pairs we need
    'funtion to create pairs of image after loading the into as array'
    datagen_args = dict(rotation_range=10,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)
    data_gen = ImageDataGenerator(**datagen_args)

    pair_images = []
    pair_labels = []
    num_classes = len(np.unique(labels)) #unique class labels
    idx = [np.where(labels == i)[0] for i in range(0,num_classes)]#taking index of data points wrt labels
    for i in range(samples):   
        for idxA in range(len(images)):
            current_img = data_gen.random_transform(images[idxA]) #random transform current image
            current_label = labels[idxA]
            #taking another image from same class
            idxB = np.random.choice(idx[current_label]) 
            posimage = data_gen.random_transform(images[idxB]) #random tranform positive image

            #append the pair of positive images with label
            pair_images.append([current_img, posimage])
            pair_labels.append([1])

            #now a negative image pair wrt to current image
            negidx = np.where(labels != current_label)[0]
            negimage = data_gen.random_transform(images[np.random.choice(negidx)]) #random transform_neg_image
            pair_images.append([current_img, negimage])
            pair_labels.append([0])
        
    return np.array(pair_images), np.array(pair_labels)
        